Работа выполнена Доронькиным Максимом. Некоторые части программы основаны на baseline решении на Stepik.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        #print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch
import numpy as np

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
!nvidia-smi
import torch
torch.cuda.is_available()

In [ ]:
import pickle
import numpy as np
import random
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline

# в sklearn не все гладко, чтобы в colab удобно выводить картинки 
# мы будем игнорировать warnings
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [ ]:
SEED = 100

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


In [ ]:
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE = 224
# работаем на видеокарте
DEVICE = torch.device("cuda")

In [ ]:
class SimpsonsDataset(Dataset):
    """
    Датасет с картинками, который паралельно подгружает их из папок
    производит скалирование и превращение в торчевые тензоры
    """
    def __init__(self, files, mode):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)
        # режим работы
        self.mode = mode

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.files)
     
        self.label_encoder = LabelEncoder()

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)

            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)
                      
    def __len__(self):
        return self.len_
      
    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image
  
    def __getitem__(self, index):
        # введем тут наши аугментации для train и val/test данных. 
        
        if self.mode == 'train': 
            transform = transforms.Compose([
                transforms.Resize(size=(RESCALE_SIZE, RESCALE_SIZE)),
                transforms.RandomRotation(degrees=45),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(p=0.25),
                transforms.ColorJitter(hue=.1, saturation=.1),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
            ])
        else:
            transform = transforms.Compose([
                transforms.Resize(size=(RESCALE_SIZE, RESCALE_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
            ])
        x = self.load_sample(self.files[index])
        x = transform(x)
        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y

In [ ]:
def imshow(inp, title=None, plt_ax=plt, default=False):
    """Imshow для тензоров"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt_ax.imshow(inp)
    if title is not None:
        plt_ax.set_title(title)
    plt_ax.grid(False)

In [ ]:
TRAIN_DIR = Path('/kaggle/input/journey-springfield/train/simpsons_dataset')
TEST_DIR = Path('/kaggle/input/journey-springfield/testset/testset')

train_val_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))


In [ ]:
from sklearn.model_selection import train_test_split

train_val_labels = [path.parent.name for path in train_val_files]
train_files, val_files = train_test_split(train_val_files, test_size=0.25, \
                                          stratify=train_val_labels)

In [ ]:
train_dataset = SimpsonsDataset(train_files, mode='train')

БЛОК ПРЕДОБРАБОТКИ ПАПКИ ТРЕНИРОВКИ. Дело в том, что наши данные очень плохо сбалансированы. Гомера Симпсона очень много, а вот некоторых персонажей всего по 3-4 картинки, значит сеть будет их забывать, ибо очень редко будет видеть, так что напишем код, чтобы их размножить. Многое в этом коде есть в открытых комментариях Stepik.

In [ ]:
def create_dct_path_labels(train_files, train_labels):
    dct_simpsons = {}
    for label_i in np.unique(train_labels).tolist():
        dct_simpsons[label_i] = []

    for path_i, label_i in zip(train_files, train_labels):
        dct_simpsons[label_i].append(path_i)

    return dct_simpsons

# Создадим словарь в котором ключами будут персонажи Симпсонов, а значениями списки с путями к картинкам.
dct_path_train = create_dct_path_labels(train_files, train_dataset.labels)

# Расширяем классы с менее 100 картинками до 120 картинок в классе. Не будем ставить больше, чтобы не перегружать GPU в будущем
for person in dct_path_train:
    if len(dct_path_train[person]) < 100:
        dct_path_train[person] = dct_path_train[person] * (100 // len(dct_path_train[person]))
        dct_path_train[person].extend(dct_path_train[person][:100 - len(dct_path_train[person])])
# Проверим что получилось 
for person in dct_path_train:
    print(f"{person}\t{len(dct_path_train[person])}")
new_train_files = []

for person in dct_path_train:
    new_train_files.extend(dct_path_train[person])

val_dataset = SimpsonsDataset(val_files, mode='val')
new_train_dataset = SimpsonsDataset(new_train_files, mode='train')

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(8, 8), \
                        sharey=True, sharex=True)
for fig_x in ax.flatten():
    random_characters = int(np.random.uniform(0,1000))
    im_val, label = new_train_dataset[random_characters]
    img_label = " ".join(map(lambda x: x.capitalize(),\
                val_dataset.label_encoder.inverse_transform([label])[0].split('_')))
    imshow(im_val.data.cpu(), \
          title=img_label,plt_ax=fig_x)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(8, 8), \
                        sharey=True, sharex=True)
for fig_x in ax.flatten():
    random_characters = int(np.random.uniform(0,1000))
    im_val, label = val_dataset[random_characters]
    img_label = " ".join(map(lambda x: x.capitalize(),\
                val_dataset.label_encoder.inverse_transform([label])[0].split('_')))
    imshow(im_val.data.cpu(), \
          title=img_label,plt_ax=fig_x)

In [ ]:
def fit_epoch(model, train_loader, criterion, optimizer):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
  
    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
              
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc

In [ ]:
def eval_epoch(model, val_loader, criterion, min_loss, eps, model_name):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0
    for inputs, labels in tqdm(val_loader):
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    #этот блок нужен для того, чтобы сохранять только самую лучшую модель по лоссу на валидации
    val_loss = running_loss / processed_size
    if val_loss < min_loss or val_loss == min_loss+eps:
        torch.save(model.state_dict(), model_name)
    val_acc = running_corrects.double() / processed_size
    return val_loss, val_acc

In [ ]:
def train(train_files, val_files, model, epochs, batch_size, model_name):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=8, shuffle=False)

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        opt = torch.optim.AdamW(model.parameters(), lr=0.001)  
        scheduler = torch.optim.lr_scheduler.StepLR(opt, 2, 0.5) #введём scheduler чтобы уменьшать learning rate динамически во время обучения
        criterion = nn.CrossEntropyLoss()
        min_loss = np.inf
        eps = 0.001
        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt)
            print("loss", train_loss)
            
            val_loss, val_acc = eval_epoch(model, val_loader, criterion, min_loss, eps, model_name)
            history.append((train_loss, train_acc, val_loss, val_acc))
            scheduler.step()
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))

    return history

In [ ]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [ ]:
from torchvision import models

В качестве рабочей модели будем использовать новшество последнего времени, разрывающее ImageNette - Efficient Net, её ещё нет в torch, поэтому загружаем её со специального github репозитория. 
Github link:https://github.com/lukemelas/EfficientNet-PyTorch/tree/master/efficientnet_pytorch

In [ ]:
pip install efficientnet_pytorch 

In [ ]:
from efficientnet_pytorch import EfficientNet
efficient_model_1 = EfficientNet.from_pretrained('efficientnet-b0')
efficient_model_1.fc = nn.Linear(1280, 42)  #модифицируем только fc слой, сделав выход на 42 класса, так как столько у нас Симпсонов, остальные слои оставляем предобученными.
efficient_model_1.to(DEVICE)

In [ ]:
batch_size = 64  #установим размер батча 64

Наконец, обучение 10 эпох efficient net на нашем датасете!

In [ ]:
history_1 = train(new_train_dataset, val_dataset, model=efficient_model_1, epochs=15, batch_size=batch_size, model_name='efficient_net_1')

In [ ]:
from efficientnet_pytorch import EfficientNet
efficient_model_2 = EfficientNet.from_pretrained('efficientnet-b0')
efficient_model_2.fc = nn.Linear(1280, 42)  #модифицируем только fc слой, сделав выход на 42 класса, так как столько у нас Симпсонов, остальные слои оставляем предобученными.
efficient_model_2.to(DEVICE)

In [ ]:
def train_2(train_files, val_files, model, epochs, batch_size, model_name):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=8, shuffle=False)

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        opt = torch.optim.AdamW(model.parameters(), lr=0.001)  
        scheduler = torch.optim.lr_scheduler.StepLR(opt, 3, 0.5) #введём scheduler чтобы уменьшать learning rate динамически во время обучения
        criterion = nn.CrossEntropyLoss()
        min_loss = np.inf
        eps = 0.001
        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt)
            print("loss", train_loss)
            
            val_loss, val_acc = eval_epoch(model, val_loader, criterion, min_loss, eps, model_name)
            history.append((train_loss, train_acc, val_loss, val_acc))
            scheduler.step()
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))

    return history

In [ ]:
history_2 = train_2(new_train_dataset, val_dataset, model=efficient_model_2, epochs=15, batch_size=batch_size*2, model_name='efficient_net_2')

А теперь загрузим сохраненную нами НАИЛУЧШУЮ версию по лоссу на валидации модели efficient net:

In [ ]:
# загружаем веса обученные на наших Симпсонах
efficient_model_1.load_state_dict(torch.load("efficient_net_1"))
efficient_model_2.load_state_dict(torch.load("efficient_net_2"))

In [ ]:
class Ensemble(nn.Module):   
    def __init__(self, modelA, modelB):
        super(Ensemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB

        self.classifier = nn.Linear(42 * 2, 42)
        
    def forward(self, x):
        x1 = self.modelA(x)
        x2 = self.modelB(x)
    
        x = torch.cat((x1, x2), dim=1)
        
        x = self.classifier(x)
        return x

In [ ]:
Ensemble = Ensemble(efficient_model_1, efficient_model_2).to(DEVICE)

In [ ]:
for param in Ensemble.parameters():
    param.requires_grad = False

for param in Ensemble.classifier.parameters():
    param.requires_grad = True

In [ ]:
history_3 = train(new_train_dataset, val_dataset, model=Ensemble, epochs=10, batch_size=batch_size, model_name='ensemble')

In [ ]:
Ensemble.load_state_dict(torch.load("ensemble"))

Ну и что теперь со всем этим делать?

Хорошо бы понять, как сделать сабмит. У нас есть сеть и методы eval у нее, которые позволяют перевести сеть в режим предсказания. Стоит понимать, что у нашей модели на последнем слое стоит softmax, которые позволяет получить вектор вероятностей того, что объект относится к тому или иному классу. Давайте воспользуемся этим.

In [ ]:
def predict_one_sample(model, inputs, device=DEVICE):
    """Предсказание, для одной картинки"""
    with torch.no_grad():
        inputs = inputs.to(device)
        model.eval()
        logit = model(inputs).cpu()
        probs = torch.nn.functional.softmax(logit, dim=-1).numpy()
    return probs

In [ ]:
random_characters = int(np.random.uniform(0,1000))
ex_img, true_label = val_dataset[random_characters]
probs_im_1 = predict_one_sample(efficient_model_1, ex_img.unsqueeze(0))
probs_im_2 = predict_one_sample(efficient_model_2, ex_img.unsqueeze(0))
#probs_im_3 = predict_one_sample(Ensemble, ex_img.unsqueeze(0))

In [ ]:
idxs = list(map(int, np.random.uniform(0,1000, 20)))
imgs = [val_dataset[id][0].unsqueeze(0) for id in idxs]

probs_ims_1 = predict(efficient_model_1, imgs)
probs_ims_2 = predict(efficient_model_2, imgs)
#probs_ims_3 = predict(Ensemble, imgs)

In [ ]:
label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))

In [ ]:
y_pred_1 = np.argmax(probs_ims_1,-1)
y_pred_2 = np.argmax(probs_ims_2,-1)
#y_pred_3 = np.argmax(probs_ims_3,-1)

actual_labels = [val_dataset[id][1] for id in idxs]

preds_class_1 = [label_encoder.classes_[i] for i in y_pred_1]
preds_class_2 = [label_encoder.classes_[i] for i in y_pred_2]
#preds_class_3 = [label_encoder.classes_[i] for i in y_pred_3]

Обратите внимание, что метрика, которую необходимо оптимизировать в конкурсе --- f1-score. Вычислим целевую метрику на валидационной выборке.

In [ ]:
from sklearn.metrics import f1_score

print(f1_score(actual_labels, y_pred_1, average='micro'))
print(f1_score(actual_labels, y_pred_2, average='micro'))
#print(f1_score(actual_labels, y_pred_3, average='micro'))

Полученная F_1 score нас бесспорно устраивает!

Сделаем классную визуализацию, чтобы посмотреть насколько сеть уверена в своих ответах. Можете исспользовать это, чтобы отлаживать правильность вывода.


import matplotlib.patches as patches
from matplotlib.font_manager import FontProperties

fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(12, 12), \
                        sharey=True, sharex=True)
for fig_x in ax.flatten():
    random_characters = int(np.random.uniform(0,1000))
    im_val, label = val_dataset[random_characters]
    img_label = " ".join(map(lambda x: x.capitalize(),\
                val_dataset.label_encoder.inverse_transform([label])[0].split('_')))
    
    

    imshow(im_val.data.cpu(), \
          title=img_label,plt_ax=fig_x)
    
    actual_text = "Actual : {}".format(img_label)
            
    fig_x.add_patch(patches.Rectangle((0, 53),86,35,color='white'))
    font0 = FontProperties()
    font = font0.copy()
    font.set_family("fantasy")
    prob_pred = predict_one_sample(efficient_model, im_val.unsqueeze(0))
    predicted_proba = np.max(prob_pred)*100
    y_pred = np.argmax(prob_pred)
    
    predicted_label = label_encoder.classes_[y_pred]
    predicted_label = predicted_label[:len(predicted_label)//2] + '\n' + predicted_label[len(predicted_label)//2:]
    predicted_text = "{} : {:.0f}%".format(predicted_label,predicted_proba)
            
    fig_x.text(1, 59, predicted_text , horizontalalignment='left', fontproperties=font,
                    verticalalignment='top',fontsize=8, color='black',fontweight='bold')

Сеть идеально уверена в своих ответах!

# **Submit на Kaggle**

In [ ]:
test_dataset = SimpsonsDataset(test_files, mode="test")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)
probs_1 = predict(efficient_model_1, test_loader)
probs_2 = predict(efficient_model_2, test_loader)
probs_3 = predict(Ensemble, test_loader)

preds_1 = label_encoder.inverse_transform(np.argmax(probs_1, axis=1))
preds_2 = label_encoder.inverse_transform(np.argmax(probs_2, axis=1))
preds_3 = label_encoder.inverse_transform(np.argmax(probs_3, axis=1))

test_filenames = [path.name for path in test_dataset.files]

In [ ]:
! ls 

Создаем data frame для submitа

In [ ]:
import pandas as pd
my_submit_1 = pd.DataFrame({'Id': test_filenames, 'Expected': preds_1})
my_submit_1.head()

In [ ]:
import pandas as pd
my_submit_2 = pd.DataFrame({'Id': test_filenames, 'Expected': preds_2})
my_submit_2.head()

In [ ]:
import pandas as pd
my_submit_3 = pd.DataFrame({'Id': test_filenames, 'Expected': preds_3})
my_submit_3.head()

НУ И ФИНАЛЬНЫЙ CSV ФАЙЛ НА САБМИТ!!!

In [ ]:
my_submit_1.to_csv('efficient_model_1_submit.csv', index=False)

In [ ]:
my_submit_2.to_csv('efficient_model_2_submit.csv', index=False)

In [ ]:
my_submit_3.to_csv('Ensemble_submit.csv', index=False)